

```
ADD YOUR PATH FROM WHERE YOU WANT TO TRAVERSE THE FILES IN "path" IN FUNCTION "test_model".

CHANGE MODEL PATH AT THE TIME OF ONLY TESTING. MODELS ARE IN SAME FOLDER AS CODE IS.
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5872 sha256=75ba9e4c1ef0092e54f71deb783737537c7c49357410b9932d52711686b4217c
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [ ]:
import os
import matplotlib.pyplot as  plt
from scipy.io import wavfile
import python_speech_features as mfcc
from sklearn import preprocessing
from scipy.io.wavfile import read
import numpy as np
import pandas as pd
import pickle
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture
import warnings
warnings.filterwarnings("ignore")



In [ ]:
def calculate_delta(array):

    rows,cols = array.shape
    # print(rows)
    # print(cols)
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas


def extract_features(audio,rate):

    mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)
    mfcc_feature = preprocessing.scale(mfcc_feature)
    # print(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta))
    return combined

## Dataset - https://drive.google.com/drive/folders/1Cxxacy_XqMsR6_x-bpN4GdTV2nc6a7KK?usp=drive_link
### Dataset Description - 65,000 one-second long utterances of 30 short words, by thousands of different people.
The SpeechCommand directory has 31 directories, one being background noise and the other 30 being the 30 different commands to be recognized. The background noise directory contains 6 files of background noise. The 30 directories contain 64 audio files each. The audio files are in .wav format. The audio files are named as follows: <speaker_id> - <utterance_number> - <emotion>.wav. The speaker_id is a unique identifier for each speaker. The utterance_number is a unique identifier for each utterance. The emotion is the emotion expressed in the utterance. The emotion labels are as follows: neutral, calm, happy, sad, angry, fearful, disgust, surprised. The emotion labels are represented numerically as follows: 0=neutral, 1=calm, 2=happy, 3=sad, 4=angry, 5=fearful, 6=disgust, 7=surprised. The audio files are 16-bit monaural audio files with a sampling rate of 16 kHz.

In training, we are making a single gmm model correspond to one class. So, in total we have 30 gmm models.

In [ ]:
alr_done = ['_background_noise_']
def train_model():
    data_dir = '/content/drive/MyDrive/KAGGLE-2/SpeechCommand'
    features = np.asarray(())
    for files in os.listdir('/content/drive/MyDrive/KAGGLE-2/SpeechCommand'):
      count = 0
      class_path = os.path.join(data_dir,files )
      if files not in alr_done:
          print(files)
          for path in os.listdir(class_path):

              sr,audio = read(os.path.join(class_path,path))
              vector  = extract_features(audio,sr)

              if features.size == 0:
                  features = vector
              else:
                  features = np.vstack((features, vector))
              count+=1
          gmm = GaussianMixture(n_components = 3, max_iter = 200, covariance_type='tied',n_init = 1)
          gmm.fit(features)
          # dumping the trained gaussian model
          picklefile = files+".gmm"
          pickle.dump(gmm,open('/content/drive/MyDrive/audiokaggle_anss/gmms/' + picklefile,'wb'))
          print(count)
          print('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)
          features = np.asarray(())
train_model()


happy
1562
+ modeling completed for speaker: happy.gmm  with data point =  (152180, 40)
dog
1566
+ modeling completed for speaker: dog.gmm  with data point =  (152168, 40)
bed
1537
+ modeling completed for speaker: bed.gmm  with data point =  (149182, 40)
tree
1541
+ modeling completed for speaker: tree.gmm  with data point =  (149627, 40)
wow
1580
+ modeling completed for speaker: wow.gmm  with data point =  (153536, 40)
cat
1567
+ modeling completed for speaker: cat.gmm  with data point =  (152125, 40)
eight
2095
+ modeling completed for speaker: eight.gmm  with data point =  (204056, 40)
no
2123
+ modeling completed for speaker: no.gmm  with data point =  (206107, 40)
go
2121
+ modeling completed for speaker: go.gmm  with data point =  (205753, 40)
right
2112
+ modeling completed for speaker: right.gmm  with data point =  (206051, 40)
nine
2105
+ modeling completed for speaker: nine.gmm  with data point =  (205913, 40)
seven
2178
+ modeling completed for speaker: seven.gmm  with dat

In [ ]:
def test_model():
    answers = {}
    modelpath = '/content/drive/MyDrive/audiokaggle_anss/gmms/'

    #model path to use when models are in same folder as code file, i.e. while testing only !!
    # modelpath = '/gmms/'

    gmm_files = [os.path.join(modelpath,fname) for fname in
                  os.listdir(modelpath) if fname.endswith('.gmm')]

    #Load the Gaussian gender Models
    models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
    speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname
                  in gmm_files]

    # Read the test directory and get the list of test audio files
    path  = "/content/drive/MyDrive/KAGGLE-2/SpeechCommandTest"



    for files in os.listdir(path):
        audio_path = os.path.join(path,files)
        # audio_path = str(input())
        sr,audio = read(audio_path)
        vector   = extract_features(audio,sr)

        log_likelihood = np.zeros(len(models))

        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()

        winner = np.argmax(log_likelihood)
        answers[files] = speakers[winner]
        # print("\tdetected as - ", speakers[winner])
    return answers

In [ ]:
answers = test_model()

In [ ]:
word_dict = {
    'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'bed': 4, 'happy': 5, 'go': 6, 'dog': 7, 'no': 8, 'wow': 9,
    'nine': 10, 'left': 11, 'stop': 12, 'three': 13, 'sheila': 14, 'one': 15, 'bird': 16, 'zero': 17, 'seven': 18,
    'up': 19, 'marvin': 20, 'two': 21, 'house': 22, 'down': 23, 'six': 24, 'yes': 25, 'on': 26, 'five': 27, 'off': 28,
    'four': 29
}

In [ ]:
answers_final = {}
for k,v in answers.items():
  answers_final[k] = word_dict[v.split("/")[-1]]
print(answers_final)

{'A_550768dd409e_nohash_1.wav': 13, 'A_603794de6a6a_nohash_3.wav': 2, 'A_5623f428ca69_nohash_0.wav': 3, 'A_6040964e8cfd_nohash_1.wav': 19, 'A_59363f170018_nohash_1.wav': 19, 'A_59624c841771_nohash_0.wav': 19, 'A_6161f5496439_nohash_0.wav': 19, 'A_60509a69672b_nohash_0.wav': 19, 'A_3044260689_nohash_0.wav': 4, 'A_5597cd85758f_nohash_0.wav': 13, 'A_486e71b4ce6_nohash_1.wav': 25, 'A_593437fc5d97_nohash_3.wav': 7, 'A_59895e3dde6b_nohash_0.wav': 6, 'A_441a2473d62_nohash_1.wav': 2, 'A_592937dca74f_nohash_3.wav': 19, 'A_172fa446c16_nohash_2.wav': 4, 'A_54975828dfa2_nohash_1.wav': 3, 'A_550668dd409e_nohash_0.wav': 13, 'A_5599d0faf7e4_nohash_0.wav': 13, 'A_60117257420c_nohash_0.wav': 19, 'A_6111cd85758f_nohash_0.wav': 19, 'A_451af7a8296_nohash_0.wav': 2, 'A_132796ac50_nohash_0.wav': 4, 'A_150d0faf7e4_nohash_0.wav': 4, 'A_5608daf230ac_nohash_2.wav': 3, 'A_5977563aa4e6_nohash_0.wav': 19, 'A_2975f6af2457_nohash_0.wav': 10, 'A_1059b3ea809_nohash_0.wav': 4, 'A_59403f2b358d_nohash_2.wav': 19, 'A_6023

In [ ]:
idmap = pd.read_csv("/content/drive/MyDrive/audiokaggle_anss/test.csv")
# idmap.head()
id_dict = idmap.set_index('AUDIO_FILE').to_dict()['ID']
updated_answers_final = {id_dict[key]: value for key, value in answers_final.items()}
answer = pd.DataFrame({'ID':updated_answers_final.keys(),'TARGET':updated_answers_final.values()})
csv = '/content/drive/MyDrive/ml_project/ans.csv'
answer.to_csv(csv,index=False)